In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor


from warnings import simplefilter

import pickle
from sklearn.externals import joblib

rand_st=4
#simplefilter("ignore") 

In [2]:
#output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx')
output=pd.read_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_minmax.xlsx')
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986714,431.864722,0.458002,0.172,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.167,Обучение на разнице в рейтинге
2,"ElasticNet(alpha=1, copy_X=True, fit_intercept...",431.367419,533.850282,0.068930,2.988,ElasticNet по сумме рейтингов
3,"ElasticNet(alpha=1, copy_X=True, fit_intercept...",191.267781,235.466242,0.425994,0.087,ElasticNet на разнице в рейтинге
4,"KNeighborsRegressor(algorithm='auto', leaf_siz...",442.258773,557.608383,0.163676,2.820,KNeighborsRegressor по сумме рейтингов
5,"KNeighborsRegressor(algorithm='auto', leaf_siz...",175.432367,222.489239,0.439167,5.564,KNeighborsRegressor на разнице в рейтинге


In [3]:
#elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,-57,4765
1,1915,1999,1957,-84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
#features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')
features=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\features_minmax.csv')

print(features.info())
print(features.shape)
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Columns: 224 entries, 0 to 223
dtypes: float64(224)
memory usage: 41.5 MB
None
(24255, 224)


,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
count,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,...,24255.000000,24255.000000,24255.000000,24255.00000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000
mean,0.544877,0.208082,0.517794,0.504664,0.503004,0.209572,0.933684,0.083822,0.087900,0.077598,...,0.239683,0.232756,0.233849,0.50923,0.236364,0.315646,0.519460,0.028389,0.041637,0.025298
std,0.417736,0.104746,0.055178,0.019608,0.093945,0.104806,0.171820,0.183491,0.081850,0.120229,...,0.309537,0.251140,0.305532,0.22908,0.251320,0.338429,0.374235,0.040491,0.119418,0.101756
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.135484,0.496158,0.504140,0.483739,0.135484,0.958946,0.009514,0.038265,0.013538,...,0.000000,0.000000,0.000000,0.37500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,0.193548,0.516303,0.504140,0.498455,0.193548,0.991838,0.016690,0.062500,0.034850,...,0.000000,0.000000,0.000000,0.50000,0.000000,0.500000,0.500000,0.014085,0.000000,0.000000
75%,1.000000,0.264516,0.539776,0.505285,0.520672,0.267742,0.996444,0.060550,0.107143,0.071691,...,0.500000,0.500000,0.500000,0.62500,0.500000,0.500000,1.000000,0.042254,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [6]:
X_train_sum, X_test_sum, Y_train_sum, Y_test_sum =ms.train_test_split(features, target_sum, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_sum), len(Y_train_sum)))
print(X_train_sum.shape)
print(Y_train_sum.shape)
print(X_test_sum.shape)
print(Y_test_sum.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


In [7]:
X_train_diff, X_test_diff, Y_train_diff, Y_test_diff =ms.train_test_split(features, target_diff, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_diff), len(Y_train_diff)))
print(X_train_diff.shape)
print(Y_train_diff.shape)
print(X_test_diff.shape)
print(Y_test_diff.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


#### Для теста

In [8]:
target_sum1=target_sum[:1000]
target_diff1=target_diff[:1000]
features1=features[:1000]

X_train1, X_test1, Y_train1, Y_test1 =ms.train_test_split(features1, target_sum1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train1), len(Y_train1)))
print(X_train1.shape)
print(Y_train1.shape)
print(X_test1.shape)
print(Y_test1.shape)

Train data size: 700 instances 
Test data size: 700 instances
(700, 224)
(700,)
(300, 224)
(300,)


In [9]:
X_train2, X_test2, Y_train2, Y_test2 =ms.train_test_split(features1, target_diff1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train2), len(Y_train2)))
print(X_train2.shape)
print(Y_train2.shape)
print(X_test2.shape)
print(Y_test2.shape)

Train data size: 700 instances 
Test data size: 700 instances
(700, 224)
(700,)
(300, 224)
(300,)


In [10]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

In [11]:
def GridTune_sum(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))

    
    grid_best_model = grid.fit(X_train1,Y_train1)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [12]:
rf_sum_params =[{'min_samples_leaf':[1,2,5],'bootstrap':[True],'oob_score':[True],
             'min_samples_split':[2,5,10,25], 'max_features':[1, 2,5,10]},
           {'min_samples_leaf':[1,2,5],'bootstrap':[False],
               'min_samples_split':[5,10,25], 'max_features':[1, 2,5,10]}]

start_rf_sum = time.time()

rf_sum=RandomForestRegressor(random_state=rand_st,n_jobs=-1,criterion= 'mae',n_estimators=200)
rf_sum_model=GridTune_sum(rf_sum_params,rf_sum,5)
rf_sum_predictions=rf_sum_model.predict(X_test1)

scores_rf_sum=predictions_report(Y_test1,rf_sum_predictions)
full_time_rf_sum=round(time.time() - start_rf_sum,3)
print(full_time_rf_sum, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'bootstrap': False, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 5}

Лучший алгоритм:
RandomForestRegressor(bootstrap=False, criterion='mae', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=5, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=-1, oob_score=False, random_state=4,
           verbose=0, warm_start=False)
MAE = 357.495
RMSE = 441.849
Correlation coefficient = 0.326
163.573 c


In [13]:
start_rf_sum = time.time()

rf_sum=RandomForestRegressor(bootstrap=False, criterion='mae', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=5, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=-1, oob_score=False, random_state=4,
           verbose=0, warm_start=False).fit(X_train_sum,Y_train_sum)

rf_sum_predictions=rf_sum.predict(X_test_sum)
scores_rf_sum=predictions_report(Y_test_sum,rf_sum_predictions)
full_time_rf_sum=round(time.time() - start_rf_sum,3)
print(full_time_rf_sum, 'c')

MAE = 351.481
RMSE = 440.278
Correlation coefficient = 0.416
215.956 c


In [14]:
output = output.append(pd.Series([str(rf_sum), scores_rf_sum[0],scores_rf_sum[1], 
                                  scores_rf_sum[2][0],full_time_rf_sum,'RandomForest по сумме рейтингов'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986714,431.864722,0.458002,0.172,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.167,Обучение на разнице в рейтинге
2,"ElasticNet(alpha=1, copy_X=True, fit_intercept...",431.367419,533.850282,0.068930,2.988,ElasticNet по сумме рейтингов
3,"ElasticNet(alpha=1, copy_X=True, fit_intercept...",191.267781,235.466242,0.425994,0.087,ElasticNet на разнице в рейтинге
4,"KNeighborsRegressor(algorithm='auto', leaf_siz...",442.258773,557.608383,0.163676,2.820,KNeighborsRegressor по сумме рейтингов
5,"KNeighborsRegressor(algorithm='auto', leaf_siz...",175.432367,222.489239,0.439167,5.564,KNeighborsRegressor на разнице в рейтинге
6,"RandomForestRegressor(bootstrap=False, criteri...",351.481084,440.277770,0.415663,215.956,RandomForest по сумме рейтингов


In [15]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_minmax.xlsx',index=False)

In [16]:
with open('rf_sum_minmax.pkl', 'wb') as f:
    pickle.dump(rf_sum,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(rf_sum, 'rf_sum1_minmax.pkl')
#model_sum= joblib.load('filename.pkl') 

['rf_sum1_minmax.pkl']

In [17]:
def GridTune_diff(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))
    
    grid_best_model = grid.fit(X_train2,Y_train2)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [18]:
rf_diff_params =[{'min_samples_leaf':[1,2,5],'bootstrap':[True],'oob_score':[True],
             'min_samples_split':[2,5,10,25], 'max_features':[1, 2,5,10]},
           {'min_samples_leaf':[1,2,5],'bootstrap':[False],
               'min_samples_split':[5,10,25], 'max_features':[1, 2,5,10]}]

start_rf_diff = time.time()

rf_diff=RandomForestRegressor(random_state=rand_st,n_jobs=-1,criterion= 'mae',n_estimators=200)
rf_diff_model=GridTune_diff(rf_diff_params,rf_diff,5)
rf_diff_predictions=rf_diff_model.predict(X_test2)

scores_rf_diff=predictions_report(Y_test2,rf_diff_predictions)
full_time_rf_diff=round(time.time() - start_rf_diff,3)
print(full_time_rf_diff, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'bootstrap': True, 'max_features': 1, 'min_samples_leaf': 5, 'min_samples_split': 25, 'oob_score': True}

Лучший алгоритм:
RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=25, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=-1, oob_score=True, random_state=4,
           verbose=0, warm_start=False)
MAE = 157.350
RMSE = 198.285
Correlation coefficient = 0.529
144.523 c


In [19]:
start_rf_diff = time.time()

rf_diff=RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=25, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=-1, oob_score=True, random_state=4,
           verbose=0, warm_start=False).fit(X_train_diff,Y_train_diff)

rf_diff_predictions=rf_diff.predict(X_test_diff)

scores_rf_diff=predictions_report(Y_test_diff,rf_diff_predictions)
full_time_rf_diff=round(time.time() - start_rf_diff,3)
print(full_time_rf_diff, 'c')

MAE = 154.751
RMSE = 196.702
Correlation coefficient = 0.561
59.2 c


In [20]:
output = output.append(pd.Series([str(rf_diff), scores_rf_diff[0],scores_rf_diff[1], 
                                  scores_rf_diff[2][0],full_time_rf_diff,'RandomForest на разнице в рейтинге'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986714,431.864722,0.458002,0.172,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.167,Обучение на разнице в рейтинге
2,"ElasticNet(alpha=1, copy_X=True, fit_intercept...",431.367419,533.850282,0.068930,2.988,ElasticNet по сумме рейтингов
3,"ElasticNet(alpha=1, copy_X=True, fit_intercept...",191.267781,235.466242,0.425994,0.087,ElasticNet на разнице в рейтинге
4,"KNeighborsRegressor(algorithm='auto', leaf_siz...",442.258773,557.608383,0.163676,2.820,KNeighborsRegressor по сумме рейтингов
5,"KNeighborsRegressor(algorithm='auto', leaf_siz...",175.432367,222.489239,0.439167,5.564,KNeighborsRegressor на разнице в рейтинге
6,"RandomForestRegressor(bootstrap=False, criteri...",351.481084,440.277770,0.415663,215.956,RandomForest по сумме рейтингов
7,"RandomForestRegressor(bootstrap=True, criterio...",154.751159,196.701782,0.560982,59.200,RandomForest на разнице в рейтинге


In [21]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_minmax.xlsx',index=False)

In [22]:
with open('rf_diff_minmax.pkl', 'wb') as f:
    pickle.dump(rf_diff,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(rf_diff, 'rf_diff1_minmax.pkl')
#model_sum= joblib.load('filename.pkl') 

['rf_diff1_minmax.pkl']

In [23]:
white_elos = (rf_sum_predictions + rf_diff_predictions)/2
black_elos = (rf_sum_predictions-rf_diff_predictions)/2
pred_elos=list(zip(white_elos,black_elos))
pred_elos[:5]

[(2197.7279999999996, 2296.078),
 (2176.72, 2302.689),
 (2190.739, 2253.597),
 (2192.5795, 2157.5575),
 (2194.397, 2115.008)]

In [24]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
        count+=2
    return round(er/count,5)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
    return round(er,5)

In [25]:
find_abs_error(pred_elos,elos),find_mean_error(pred_elos,elos)

(71803.074, 4.93356)